In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#Main
from kaggle.competitions import nflrush

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import the necessary packages
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

maxo = 30
mino = -5
#intero = (maxo - mino) + 1
intero = 1
gepoch = 50
galpha=0.0005
glayer1 = 225
glayer2 = 125
glayer3 = 25
#dictionary
gcleanedListCat = {}


In [ ]:
class NeuralNetwork:
	def __init__(self, layers, alpha=0.1):
		# initialize the list of weights matrices, then store the
		# network architecture and learning rate
		self.W = []
		self.layers = layers
		self.alpha = alpha

		# start looping from the index of the first layer but
		# stop before we reach the last two layers
		for i in np.arange(0, len(layers) - 2):
			# randomly initialize a weight matrix connecting the
			# number of nodes in each respective layer together,
			# adding an extra node for the bias
			w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
			self.W.append(w / np.sqrt(layers[i]))

		# the last two layers are a special case where the input
		# connections need a bias term but the output does not
		w = np.random.randn(layers[-2] + 1, layers[-1])
		self.W.append(w / np.sqrt(layers[-2]))

	def __repr__(self):
		# construct and return a string that represents the network
		# architecture
		return "NeuralNetwork: {}".format(
			"-".join(str(l) for l in self.layers))

	def sigmoid(self, x):
		# compute and return the sigmoid activation value for a
		# given input value
		return 1.0 / (1 + np.exp(-x))

	def sigmoid_deriv(self, x):
		# compute the derivative of the sigmoid function ASSUMING
		# that `x` has already been passed through the `sigmoid`
		# function
		return x * (1 - x)
    
	def swish(self, x):
		# compute and return the sigmoid activation value for a
		# given input value
		return x * self.sigmoid(x)

	def swish_deriv(self, x):
		return (np.exp(-x) * (x + 1) + 1) / (1 + np.exp(-x))**2

	def fit(self, X, y, epochs=1000, displayUpdate=100):
		# insert a column of 1's as the last entry in the feature
		# matrix -- this little trick allows us to treat the bias
		# as a trainable parameter within the weight matrix
		X = np.c_[X, np.ones((X.shape[0]))]

		# loop over the desired number of epochs
		for epoch in np.arange(0, epochs):
			# loop over each individual data point and train
			# our network on it
			for (x, target) in zip(X, y):
				self.fit_partial(x, target)

			# check to see if we should display a training update
			if epoch == 0 or (epoch + 1) % displayUpdate == 0:
				loss = self.calculate_loss(X, y)
				print("[INFO] epoch={}, loss={:.7f}".format(
					epoch + 1, loss))

	def fit_partial(self, x, y):
		# construct our list of output activations for each layer
		# as our data point flows through the network; the first
		# activation is a special case -- it's just the input
		# feature vector itself
		A = [np.atleast_2d(x)]

		# FEEDFORWARD:
		# loop over the layers in the network
		for layer in np.arange(0, len(self.W)):
			# feedforward the activation at the current layer by
			# taking the dot product between the activation and
			# the weight matrix -- this is called the "net input"
			# to the current layer
			net = A[layer].dot(self.W[layer])
			lastlayer = len(self.W)-1
			if (layer == lastlayer):
				# derniere couche
				out = self.swish(net)
				#print('out')
				#print(out)
				#print(net)                
			else:
				# computing the "net output" is simply applying our
				# non-linear activation function to the net input
				out = self.sigmoid(net)
				#print('out')
				#print(out)
				#print(net) 
			# once we have the net output, add it to our list of
			# activations
			A.append(out)

		# BACKPROPAGATION
		# the first phase of backpropagation is to compute the
		# difference between our *prediction* (the final output
		# activation in the activations list) and the true target
		# value
		error = A[-1] - y

		#print('error')
		#print(error)
		#print(y)
		#print(A[-1])
        
		# from here, we need to apply the chain rule and build our
		# list of deltas `D`; the first entry in the deltas is
		# simply the error of the output layer times the derivative
		# of our activation function for the output value
		D = [error * self.swish_deriv(A[-1])]

		# once you understand the chain rule it becomes super easy
		# to implement with a `for` loop -- simply loop over the
		# layers in reverse order (ignoring the last two since we
		# already have taken them into account)
		for layer in np.arange(len(A) - 2, 0, -1):
			# the delta for the current layer is equal to the delta
			# of the *previous layer* dotted with the weight matrix
			# of the current layer, followed by multiplying the delta
			# by the derivative of the non-linear activation function
			# for the activations of the current layer
			delta = D[-1].dot(self.W[layer].T)
			delta = delta * self.sigmoid_deriv(A[layer])
			D.append(delta)

		# since we looped over our layers in reverse order we need to
		# reverse the deltas
		D = D[::-1]

		# WEIGHT UPDATE PHASE
		# loop over the layers
		for layer in np.arange(0, len(self.W)):
			# update our weights by taking the dot product of the layer
			# activations with their respective deltas, then multiplying
			# this value by some small learning rate and adding to our
			# weight matrix -- this is where the actual "learning" takes
			# place
			self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

	def predict(self, X, addBias=True):
		# initialize the output prediction as the input features -- this
		# value will be (forward) propagated through the network to
		# obtain the final prediction
		p = np.atleast_2d(X)

		# check to see if the bias column should be added
		if addBias:
			# insert a column of 1's as the last entry in the feature
			# matrix (bias)
			p = np.c_[p, np.ones((p.shape[0]))]

		# loop over our layers in the network
		for layer in np.arange(0, len(self.W)):
			# computing the output prediction is as simple as taking
			# the dot product between the current activation value `p`
			# and the weight matrix associated with the current layer,
			# then passing this value through a non-linear activation
			# function
			#p = self.sigmoid(np.dot(p, self.W[layer]))
			lastlayer = len(self.W)-1
			if (layer == lastlayer):
				# derniere couche
				p = self.swish(np.dot(p, self.W[layer]))
			else:
				# computing the "net output" is simply applying our
				# non-linear activation function to the net input
				p = self.sigmoid(np.dot(p, self.W[layer]))
		# return the predicted value
		return p

	def calculate_loss(self, X, targets):
		# make predictions for the input data points then compute
		# the loss
		targets = np.atleast_2d(targets)
		predictions = self.predict(X, addBias=False)
		loss = 1/len(X) * np.sum((predictions - targets) ** 2)

		# return the loss
		return loss


In [ ]:
def GetYardEncoded(yards, onehot_encoded):  
    index = yards - mino
    
    if (index > (maxo - mino)) : 
        index = maxo - mino 
    if (index < 0) : 
        index = 0
    
    #for number in range(index + 1, intero, 1):
    #    onehot_encoded[index][number] = 1
    #print(onehot_encoded[index])
    
    return onehot_encoded[index]

def GetYardDecoded(yards):  
   
    for index, val in enumerate(yards, mino):
        if val == 1 :
            yardsGain = index
            break
 
    return yardsGain

def getY(ftrain_df):
   
    #even_list = list(range(mino,maxo+1))
    #even_list = array(even_list)  

    # binary encode
    #onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    #even_list = even_list.reshape(len(even_list), 1)
    #onehot_encoded = onehot_encoder.fit_transform(even_list)

    #Construit toutes les réponses possibles
    listy = list()
    for number in ftrain_df.Yards:
        listy.append(number)
    y = np.array(listy)  
    return y

def ComputeDefenderStats(ftrain_df, keepyard):
    from sklearn.preprocessing import MinMaxScaler
    #distance to intercept
    
def ComputeOffenseStats(ftrain_df, keepyard):
    from sklearn.preprocessing import MinMaxScaler
    # number of blocker in front
        
def FormatTrainSet(ftrain_df, keepyard, min_max_scaler, isfit, traincomplet):
          
    ftrain_dfOther = ftrain_df[ftrain_df['NflId'] != ftrain_df['NflIdRusher']]
    #filter row
    ftrain_df = ftrain_df[ftrain_df['NflId'] == ftrain_df['NflIdRusher']]
    other = ftrain_dfOther[[ 'X', 'Y', 'S', 'A', 'Dir', 'Orientation']]
    arrother = np.array(other)
    arrother = np.reshape(arrother, (-1, 126))
    ftrain_df = ftrain_df.reset_index(drop=True)
    dfother=pd.DataFrame(arrother, columns = [str(i) for i in range(0,126)])
    ftrain_df = pd.concat([ftrain_df, dfother], axis=1)
    listcol = list([str(i) for i in range(0,126)])
    list_one = [ 'X', 'Y', 'S', 'A', 'Dis', 'Dir', 'Position', 'PlayerWeight', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'Team', 'Orientation', 'YardLine',  'PossessionTeam', 'Down', 'Distance', 'DefendersInTheBox', 'OffenseFormation', 'Yards']    
    list_one_noyard = [ 'X', 'Y', 'S', 'A', 'Dis', 'Dir', 'Position', 'PlayerWeight', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'Team', 'Orientation', 'YardLine',  'PossessionTeam', 'Down', 'Distance', 'DefendersInTheBox', 'OffenseFormation']
    merged_list = list_one + listcol
    normalise_list = [ 'X', 'Y', 'S', 'A', 'Dis', 'Dir', 'PlayerWeight', 'Orientation', 'YardLine', 'Down', 'Distance', 'DefendersInTheBox', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay' ]    
    normalise_list = normalise_list + listcol
    
    #print(merged_list)
    #print(list(ftrain_df.columns)) 
    
    if keepyard :
        #Keep only some column
        ftrain_df = ftrain_df[merged_list]
        #ftrain_df = ftrain_df[[ 'X', 'Y', 'S', 'A', 'Team', 'Orientation', 'YardLine',  'PossessionTeam', 'Down', 'Distance', 'DefendersInTheBox', 'OffenseFormation', 'Yards']] 
    else :
        merged_list = list_one_noyard + listcol
        ftrain_df = ftrain_df[merged_list]
        #Keep only some column
        #ftrain_df = ftrain_df[[ 'X', 'Y', 'S', 'A', 'Team', 'Orientation', 'YardLine',  'PossessionTeam', 'Down', 'Distance', 'DefendersInTheBox',  'OffenseFormation']]
    
    column_names_to_normalize = normalise_list 
    x = ftrain_df[column_names_to_normalize].values
    if isfit:
        x_scaled = min_max_scaler.transform(x)
    else : 
        x_scaled = min_max_scaler.fit_transform(x)
        
    df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = ftrain_df.index)
     
    #warning here
    ftrain_df[column_names_to_normalize] = df_temp 
    ftrain_df = ftrain_df.dropna()
    ftrain_df = ftrain_df.reset_index(drop=True)
    
    if isfit:
        x_scaled = min_max_scaler.transform(x)
        for col in ftrain_df.select_dtypes(include=[np.object]).columns:
            ftrain_df[col] = pd.Categorical(ftrain_df[col], categories = gcleanedListCat[col])
    else : 
        x_scaled = min_max_scaler.fit_transform(x)  
        for col in ftrain_df.select_dtypes(include=[np.object]).columns:
            gcleanedListCat[col] = [x for x in traincomplet[col].unique() if x == x]
            ftrain_df[col] = pd.Categorical(ftrain_df[col], categories = gcleanedListCat[col])
    
    ftrain_df = pd.get_dummies(ftrain_df)

    return ftrain_df

def train_my_model(train_df, nn, y):
    X = np.array(train_df)
    #print(X)
    nn.fit(X, y, gepoch, 1)

def add_row(df, row):
    df.loc[-1] = row
    df.index = df.index + 1  
    return df.sort_index()

def make_my_predictionstest(test_df, sample_prediction_df, train_df2, scaler):
    test = FormatTrainSet(test_df, False, scaler, True, train_df2)
    print(test)
    

def make_my_predictions(test_df, sample_prediction_df, train_df2, scaler):
   
    test_df = FormatTrainSet(test_df, False, scaler, True, train_df2)
    
    X = np.array(test_df)
    
    # now that our network is trained, loop over the XOR data points
    for (x) in X:
        # make a prediction on the data point and display the result
        # to our console
        pred = nn.predict(x)
        pred = pred[0]
        numyard = pred[0]

        print('Prediction')
        numyard = round(numyard-1.6)
        print(numyard)
        
        even_list = np.zeros(199)
        index = numyard + 99
        
        for idx, val in enumerate(even_list, 0):
            if idx >= index :
                even_list[idx] = 1
        
        sample_prediction_df.iloc[0] = even_list
        #print(sample_prediction_df)
        
    return sample_prediction_df 



In [ ]:
# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

#filter row
min_max_scaler = MinMaxScaler()
train_df = FormatTrainSet(train_df, True, min_max_scaler, False, train_df)
y = getY(train_df)
train_df = train_df.drop(columns=['Yards'])

alpha=galpha
nn = NeuralNetwork([len(train_df.columns), glayer1, glayer2, intero], alpha)
#nn = NeuralNetwork([len(train_df.columns), glayer1, glayer2, glayer3, intero], alpha)
#nn = NeuralNetwork([len(train_df.columns), 300, 175, 125, intero], alpha)
#nn = NeuralNetwork([len(train_df.columns), 77, 88, intero], alpha)
train_my_model(train_df, nn, y)

#official
train_df2 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

env = nflrush.make_env()
for (test_df, sample_prediction_df) in env.iter_test():
    predictions_df = make_my_predictions(test_df, sample_prediction_df,train_df2, min_max_scaler)
    env.predict(predictions_df)
env.write_submission_file()